Notebook can be found at [probprog-ai-tuwien/turing-time-series](https://github.com/probprog-ai-tuwien/turing-time-series).

In [ ]:
using Turing
using StatsPlots
using Optim

In [ ]:
PRINT_PROGRESS = false
include("read_data.jl")
include("utils.jl")

# Iterative Bayesian Modeling with Probabilistic Programming

- ## Time Series Model
- ## Trend
- ## Seasonality
- ## Debugging
- ## Combine + Forecast
- ## Bonus

### Dataset: World-Wide Monthly Air Passengers

Data source: https://www.transtats.bts.gov/Data_Elements.aspx

In [ ]:
base_plot(air_passengers_2013_2018)

### First Start: A Linear Trend Model

We start easy by modelling the trend of the time series.

A Bayesian linear regression is the "hello world" example for probabilistic programs and can be used to model linear trends.

In [ ]:
@model function trend_model_1(x::Vector{Float64}, y::Vector{Float64})
    # priors are guided by data but chosen to be very broad
    # (we do not want to favour any specific values)
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    # error models how noisy the data is, it has to be >0 => InverseGamma
    error ~ InverseGamma(2,3)
    
    for i in eachindex(y)
        # shift the x-axis by start year 0...16 instead of 2003...2019
        y[i] ~ Normal(slope * (x[i] - x[1]) + intercept, error+1e-5)
    end
end

We focus on the part of the time series which approximately exhibits a linear trend.

In [ ]:
upwards_trend_data = air_passengers_2013_2018[2009 .<= air_passengers_2013_2018[!,"Year"],:]
base_plot(upwards_trend_data)

If the model allows it (continuous density), the inference algorithm `NUTS` is always a good first choice.

In [ ]:
Turing.Random.seed!(0)
res_1 = sample(trend_model_1(upwards_trend_data[!,"Date"], upwards_trend_data[!,"Total"]), NUTS(), 3000, progress=PRINT_PROGRESS)
res_1

While we can inspect the summary statistics above, plotting the results gives a quick and easy check if everything works as expected.

To start, we plot the maximum-a-posteriori (MAP) estimate - the most likely values for the latent variables under the posterior.

In [ ]:
function plot_trend_model_1_map(data, slope, intercept, error)
    p = base_plot(data)
    t_min = data[!,"Date"][1]
    plot!(p, t -> slope * (t-t_min) + intercept, color="red")
    plot!(p, t -> slope * (t-t_min) + intercept + sqrt(error), color="orange")
    plot!(p, t -> slope * (t-t_min) + intercept - sqrt(error), color="orange")
    p
end
_, map_vector = get_map(res_1)
plot_trend_model_1_map(upwards_trend_data, map_vector...)

But of course as Bayesian statisticians we also care about the uncertainty around our estimates.

So we plot samples from the posterior.

In [ ]:
function plot_trend_model_1_posterior(data, res)
    p = base_plot(data)
    t_min = data[!,"Date"][1]
    for i in 1:length(res)
        _, trace = get_trace(res, i)
        slope, intercept, error = trace
        plot!(p, t -> slope * (t-t_min) + intercept, color="black", alpha=0.05)
    end
    p
end
plot_trend_model_1_posterior(upwards_trend_data, res_1[1:30:end])

### Changing Trend

From the plot above we can see that a linear model does not perfectly describe the trend in the time series.

In particular, after 2014 there seems to be a steeper increase in passengers then before.

Lets see if we can model this with a changepoint.

In [ ]:
function trend_model_2_f(t::Real, slope::Real, intercept::Real, changepoint::Real, adjustment::Real)
    k = slope
    m = intercept
    # UPGRADE: if the time series exeeds the changepoint, we adjust slope and intercept in a continuous way
    if changepoint ≤ t
        k += adjustment
        m -= changepoint * adjustment
    end
    return k * t + m
end

@model function trend_model_2(x::Vector{Float64}, y::Vector{Float64})
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(2,3)

    # UPGRADE: sample changepoint and adjustment to be made at changepoint
    changepoint ~ Uniform(0, x[end] - x[1])
    adjustment ~ Normal(0,1)

    for i in eachindex(y)
        y[i] ~ Normal(trend_model_2_f(x[i]-x[1], slope, intercept, changepoint, adjustment), error+1e-5)
    end
end

In [ ]:
Turing.Random.seed!(0)
res_2 = sample(trend_model_2(upwards_trend_data[!,"Date"], upwards_trend_data[!,"Total"]), NUTS(), 3000, progress=PRINT_PROGRESS);

Again, plotting is our best friend.

In [ ]:
function plot_trend_model_2(data, slope, intercept, error, changepoint, adjustment)
    p = base_plot(data)
    t_min = data[!,"Date"][1]
    plot!(p, t -> trend_model_2_f(t - t_min, slope, intercept, changepoint, adjustment), color="red")
    plot!(p, t -> trend_model_2_f(t - t_min, slope, intercept, changepoint, adjustment) + sqrt(error), color="orange")
    plot!(p, t -> trend_model_2_f(t - t_min, slope, intercept, changepoint, adjustment) - sqrt(error), color="orange")
    vline!([changepoint + t_min], linestyle=:dash, color="black")
    p
end
_, map_vector = get_map(res_2)
single_chainpoint_res = res_2
plot_trend_model_2(upwards_trend_data, map_vector...)

### Multiple Changepoints

If we turn our attention back to the full time series, we can see that there seem to be multiple changes in the trend.

In [ ]:
base_plot(air_passengers_2013_2018)

Let's see if we can simply modify our model to handle multiple changepoints.

In [ ]:
function trend_model_3_1_f(t::Real, slope::Real, intercept::Real, changepoints::Vector{<:Real}, adjustments::Vector{<:Real})
    k = slope
    m = intercept
    # UPGRADE: now iterate over multiple changepoints
    for (changepoint, adjustment) in zip(changepoints, adjustments)
        if changepoint ≤ t
            k += adjustment
            m -= changepoint * adjustment
        end
    end
    return k * t + m
end

@model function trend_model_3_1(x::Vector{Float64}, y)
    slope ~ Normal(0,1)
    intercept ~ Normal(6,1)
    error ~ InverseGamma(2,3)
    
    # UPGRADE: sample the number of changepoints and multiple changepoint candidates
    n_changepoints ~ DiscreteUniform(1,5)
    changepoints ~ filldist(Uniform(0, x[end] - x[1]), 5)
    adjustments ~ filldist(Normal(0,1), 5)
    
    k = slope
    m = intercept
    j = 1
    
    if ismissing(y)
        y = zeros(length(x))
    end
    for i in eachindex(y)
        y[i] ~ Normal(trend_model_3_1_f(x[i]-x[1], slope, intercept, changepoints[1:n_changepoints], adjustments[1:n_changepoints]), error+1e-5)
    end
end

In [ ]:
try
    Turing.Random.seed!(0)
    res_3_1 = sample(trend_model_3_1(air_passengers_2013_2018[!,"Date"], air_passengers_2013_2018[!,"Total"]), NUTS(), 3000, progress=PRINT_PROGRESS)
catch e
    println(e)
end

`NUTS` does not work anymore because we have the discrete variable `n_changepoints`.

Let's give it a try with simple metroplis hastings `MH`.

In [ ]:
Turing.Random.seed!(0)
sampler_3_1 = MH()
res_3_1 = sample(trend_model_3_1(air_passengers_2013_2018[!,"Date"], air_passengers_2013_2018[!,"Total"]), sampler_3_1, 3000, progress=PRINT_PROGRESS);

Unfortunately, plotting shows that the inference was not able to produce the correct changepoints.

In [ ]:
function plot_trend_model_3_1(data, slope, intercept, error, changepoints, adjustments)
    p = base_plot(data)
    x = data isa Tuple ? data[1] : data[!,"Date"]
    t_min = x[1]
    plot!(p, t -> trend_model_3_1_f(t - t_min, slope, intercept, changepoints, adjustments), color="red")
    plot!(p, t -> trend_model_3_1_f(t - t_min, slope, intercept, changepoints, adjustments) + sqrt(error), color="orange")
    plot!(p, t -> trend_model_3_1_f(t - t_min, slope, intercept, changepoints, adjustments) - sqrt(error), color="orange")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    return p
end
_, map_vector = get_map(res_3_1)
n_changepoints = Int(map_vector[4])
changepoints = map_vector[5:9][1:n_changepoints]
adjustments = map_vector[10:end][1:n_changepoints]

plot_trend_model_3_1(air_passengers_2013_2018, map_vector[1], map_vector[2], map_vector[3], changepoints, adjustments)

### Debugging a Probabilistic Program

The first approach to debugging inference of a probabilistic program is to look at the MCMC diagnostic plots.

They show how the values of the latent variables are explored during inference.

We can see that the values hardly change when using `MH` => bad exploration of the state space.

In [ ]:
plot(res_3_1[["slope", "intercept", "n_changepoints", "changepoints[1]", "adjustments[1]"]])

For reference, below you can see how MCMC diagnostic plots are supposed to look if the inference algorithm explores the state space well.

In [ ]:
plot(single_chainpoint_res[["slope", "intercept", "changepoint", "adjustment"]])

There are two reasons why the inference may fail like in our case:
- the model does not describe the data well
- the inference algorithm is not able to compute the correct posterior

We can test for the first reason by inspecting the "worlds" our model can generate - this is called a **Prior Predictive Check**.

To test for the second reason, we try our inference algorithm on simulated data, because then we know that the model perfectly describes the data - this is called **Fake Data Simulation**.

#### Prior Predictive Check

In [ ]:
Turing.Random.seed!(0)
x = air_passengers_2013_2018[!,"Date"]

# use probalistic program as a generative model
res = sample(trend_model_3_1(x, missing), Prior(), 19, progress=false)

t_min = x[1]
t_max = x[end]

p = plot(x, air_passengers_2013_2018[!,"Total"], legend=false, xticks=[], yticks=[])
ps = [p]
for i in 1:length(res)
    _, trace = get_trace(res,i)
    slope, intercept = trace[1:2]
    n_changepoints = Int(trace[4])
    changepoints = trace[5:9][1:n_changepoints]
    adjustments = trace[10:14][1:n_changepoints]
    y_from_prior = trace[15:end]
    p = plot(x, t -> trend_model_3_1_f(t - t_min, slope, intercept, changepoints, adjustments), color=:gray, legend=false, xticks=[])#, yticks=[])
    plot!(p, x, y_from_prior)
    
    cps = changepoints .+ t_min
    vline!(p, cps[(t_min .< cps) .& (cps .< t_max)], linestyle=:dash)
    push!(ps, p)
end
plot(ps..., layout=(5,4), size=(1000,1000))

The generated "worlds" seem to agree with our intention for modelling changepoints. 

#### Fake-Data Simulation

In [ ]:
ps = []
for i in 1:12
    # for each of the simulated data sets ...
    _, trace = get_trace(res,i)
    slope, intercept = trace[1:2]
    n_changepoints = Int(trace[4])
    true_changepoints = trace[5:9][1:n_changepoints]
    adjustments = trace[9:14][1:n_changepoints]
    y_from_prior = trace[15:end]

    # ... perform posterior inference ...
    Turing.Random.seed!(10)
    ppc_res = sample(trend_model_3_1(x, y_from_prior), sampler_3_1, 3000, progress=false)

    # ... and check if we can recover the true latents
    _, map_vector = get_map(ppc_res)
    n_changepoints = Int(map_vector[4])
    changepoints = map_vector[5:9][1:n_changepoints]
    adjustments = map_vector[9:14][1:n_changepoints]

    p = plot_trend_model_3_1((x, y_from_prior), map_vector[1], map_vector[2], map_vector[3], changepoints, adjustments)
    vline!(true_changepoints .+ t_min, linestyle=:dash, color=:green)
    plot!(p, xticks=[], yticks=[], xlabel="", ylabel="")
    
    push!(ps,p)
end
plot(ps..., layout=(3,4), size=(1000,600))

We conclude that the model should be able to describe our data, however, the inference algorithm is not powerful enough.

#### Multiple Changepoints - Second Try

Instead of trying to engineer a powerful inference algorithm, we adjust the model slightly to work with `NUTS` again.

The problem was that the number of true changepoints is discrete and unknown.

We now fix the changepoints at the beginning of each year, but put a restrictive prior on the adjustments,  
such that the adjustments tend to 0 if there is no real change in trend.

In [ ]:
@model function trend_model_3(x::Vector{Float64}, y::Vector{Float64})
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(2,3)
    tau ~ InverseGamma(1,1)

    # UPGRADE: fix one changepoint per year, put restrictive Laplace prior on adjustments
    n_changepoints = length(y) ÷ 12
    adjustments ~ filldist(Laplace(0,tau+1e-5), n_changepoints)
    
    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        # UPGRADE: adjust slope and intercept at beginning of each year
        if i % 12 == 1
            k += adjustments[j]
            m -= (x[i] - x[1]) * adjustments[j]
            j += 1
        end
        y[i] ~ Normal(k * (x[i] - x[1]) + m, error+1e-5)
    end
end

For even faster model iteration, we may solve directly for the MAP which is faster.

In [ ]:
# Turing.Random.seed!(0)
# res_3 = sample(trend_model_3(air_passengers_2013_2018[!,"Date"], air_passengers_2013_2018[!,"Total"]), NUTS(), 3000, progress=PRINT_PROGRESS);

In [ ]:
res_3 = optimize(trend_model_3(air_passengers_2013_2018[!,"Date"], air_passengers_2013_2018[!,"Total"]), MAP(), Optim.Options(iterations=10_000))

In [ ]:
function trend_model_3_f(t, slope, intercept, changepoints, adjustments)
    ix = changepoints .<= t
    return (slope + sum(adjustments[ix])) * t + (intercept - changepoints[ix]'adjustments[ix])
end

function plot_trend_model_3(data, slope, intercept, error, adjustments)
    p = base_plot(data)
    x = data[!,"Date"]
    t_min = x[1]
    changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
    plot!(p, t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments), color="red")
    plot!(p, t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments) + sqrt(error), color="orange")
    plot!(p, t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments) - sqrt(error), color="orange")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    p
end
# _, map_vector = get_map(res_3)
map_vector = res_3.values.array
plot_trend_model_3(air_passengers_2013_2018, map_vector[1], map_vector[2], map_vector[3], map_vector[5:end])

Now we are happy with the trend model.

### Seasonality

So far we have completely neglected the seasonality of the time series.

But having modelled the trend of the time series accurately, we can substract the trend from the data and solely focus on the seasonality.

In [ ]:
x = air_passengers_2013_2018[!,"Date"]
y = air_passengers_2013_2018[!,"Total"]
t_min = x[1]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
slope, intercept, error, adjustments = map_vector[1], map_vector[2], map_vector[3], map_vector[5:end]

# substract the trend model from the data
y_stationary =  y .- map(t -> trend_model_3_f(t - t_min, slope, intercept, changepoints, adjustments), x)

base_plot((x, y_stationary))
title!("air passengers without trend")
ylabel!("")

Seasonality is modelled by decomposing the time series into sinusoidal functions with different frequencies and phase shifts.

In [ ]:
function seasonality_component(t::Float64, N_frequencies::Int, beta::Vector{<:Real})
    s = 0
    for n in 1:N_frequencies
        s += beta[2*n-1] * sin(2*pi*n*t)
        s += beta[2*n] * cos(2*pi*n*t)
    end
    return s
end

@model function seasonality_model(x::Vector{Float64}, y::Vector{Float64}, N_frequencies::Int)
    beta ~ filldist(Normal(0,1.), 2*N_frequencies)
    error ~ InverseGamma(2,3)
    
    for i in eachindex(y)
        t = x[i] - x[1]
        s = seasonality_component(t, N_frequencies, beta)
        y[i] ~ Normal(s, error+1e-5)
    end

end

Try out different number of frequencies.

In [ ]:
ps = []
for N_frequencies in 1:4
    res = optimize(seasonality_model(x, y_stationary, N_frequencies), MAP())
    beta = res.values.array

    mask = 1:60
    p = plot(x[mask], y_stationary[mask], label="air passengers", legend=:topleft, title="Number of frequencies: $N_frequencies")
    plot!(t -> seasonality_component(t, N_frequencies, beta), label="seasonality model MAP")
    push!(ps, p)
end
plot(ps...,layout=(2,2), size=(1200,800))

In [ ]:
N_frequencies = 3 # seems to be a good choice

#### Combining the Trend and Seasonality Components

Simply combine trend and seasonality models.

In [ ]:
@model function prophet(x::Vector{Float64}, y::Vector{Float64}, N_frequencies::Int)
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(2,3)
    n_changepoints = Int(ceil(length(y) / 12)) # changepoint at each year
    tau ~ InverseGamma(1,1)
    adjustments ~ filldist(Laplace(0,tau+1e-5), n_changepoints)
    beta ~ filldist(Normal(0,1.), 2*N_frequencies)

    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        t = (x[i] - x[1])
        if i % 12 == 1
            k += adjustments[j]
            m -= t * adjustments[j]
            j += 1
        end
        s = seasonality_component(t, N_frequencies, beta)

        y[i] ~ Normal(k * t + m + s, error+1e-5)
    end
end

Solve for the MAP.

In [ ]:
N_frequencies = 3
x = air_passengers_2013_2018[!,"Date"]
y = air_passengers_2013_2018[!,"Total"]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]

Turing.Random.seed!(1)
map_estimate = optimize(prophet(x, y, N_frequencies), MAP(), Optim.Options(iterations=10000))

In [ ]:
function prophet_model_f(t, slope, intercept, changepoints, adjustments, N_frequencies, beta)
    return trend_model_3_f(t, slope, intercept, changepoints, adjustments) + seasonality_component(t, N_frequencies, beta)
end

function plot_prophet_model(data, slope, intercept, error, adjustments, N_frequencies, beta)
    p = base_plot(data)
    x = data[!,"Date"]
    t_min = x[1]
    changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
    plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta), color="red")
    # plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta) + sqrt(error), color="orange")
    # plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta) - sqrt(error), color="orange")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    p
end

In [ ]:
map_vector = map_estimate.values.array
n_changepoints = length(y) ÷ 12

slope, intercept, error, tau,  adjustments = map_vector[1:4]..., map_vector[5:(5+n_changepoints-1)]
beta = map_vector[5+n_changepoints : end]

plot_prophet_model(air_passengers_2013_2018, slope, intercept, error, adjustments, N_frequencies, beta)

We can be quite happy with the fit of the model.

### Forecasting

Of course, we are also interested in forecasting the time series.

To get accurate uncertainty quantification, we go back to our full Bayesian approach:   
We approximate the full posterior instead of only optimising for the MAP.

However, it is a good idea to set the MAP as starting point for the inference process.

In [ ]:
Turing.Random.seed!(0)
res_prophet = sample(prophet(x, y, N_frequencies), NUTS(), 1000, progress=PRINT_PROGRESS, init_params=map_estimate.values.array);

Next, we extrapolate into the future by continuing the trend, respecting the seasonality and randomly sampling future changepoints.

In [ ]:
function prophet_forecast(x::Vector{Float64}, y::Vector{Float64},
    m::Float64, k::Float64, error::Float64, N_frequencies::Int, beta::Vector{Float64},
    forecast::Int)

    tau = mean(abs, adjustments)

    n_future_changepoints = ((length(y) + forecast) ÷ 12) - (length(y) ÷ 12)
    future_adjustments = rand(filldist(Laplace(0,tau), n_future_changepoints))

    y_pred = zeros(forecast)
    Δ = x[2] - x[1]
    x_future = x[end]
    i = length(y)+1
    j = 1
    while i <= length(y) + forecast
        x_future += Δ
        t = (x_future - x[1])

        if i % 12 == 1
            k += future_adjustments[j]
            m -= t * future_adjustments[j]
            j += 1
        end

        s = seasonality_component(t, N_frequencies, beta)
        y_pred[i-length(y)] = rand(Normal(k * t + m + s, error))
        i += 1
    end

    return y_pred
end

We write a helper function to extrapolate randomly `n_samples_per_trace` times for each sample in the posterior.

In [ ]:
function sample_forecast(res, changepoints, forecast, n_samples_per_trace)
    n_changepoints = length(changepoints)
    y_pred = zeros(n_samples_per_trace * length(res), forecast)

    j = 0
    for i in 1:length(res)
        trace = res[i].value.data

        slope, intercept, error, tau,  adjustments = trace[1:4]..., trace[5:(5+n_changepoints-1)]
        beta = trace[5+n_changepoints : end]

        for _ in 1:n_samples_per_trace
            j += 1
            k = slope + sum(adjustments)
            m = intercept - adjustments'changepoints
            y_pred[j,:] = prophet_forecast(x,y,m,k,error,N_frequencies,beta,forecast)
        end
    end
    return y_pred
end

Again, with plotting the forecast, we can check if we are satisfied with the results.

In [ ]:
function plot_forecast(x, y, x_forecast, y_pred)
    forecast = length(x_forecast)
    p = plot(x, y, label="air passenger historic data", xlabel="year", ylabel="passenger [10^7]", legend=:topleft)
    q05 = map(i -> quantile(y_pred[:,i], 0.05), 1:forecast)
    q50 = map(i -> quantile(y_pred[:,i], 0.5), 1:forecast)
    q95 = map(i -> quantile(y_pred[:,i], 0.95), 1:forecast)
    plot!(x_forecast, q50, ribbon=(q50-q05,q95-q50), label="forecast")
    return p
end

In [ ]:
n_year_forecast = 4
forecast = n_year_forecast * 12
x_forecast = collect(maximum(x) .+ (1:forecast)./12)

y_pred = sample_forecast(res_prophet, changepoints, forecast, 10)
plot_forecast(x, y, x_forecast, y_pred)

### BONUS: How to model the pandemic ....

In [ ]:
base_plot(air_passengers_2013_2023)

What happens if we apply the unmodified model?

In [ ]:
N_frequencies = 3
x = air_passengers_2013_2023[!,"Date"]
y = air_passengers_2013_2023[!,"Total"]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]

Turing.Random.seed!(0)
map_estimate = optimize(prophet(x, y, N_frequencies), MAP(), Optim.Options(iterations=10000))

Turing.Random.seed!(0)
res_prophet_2023 = sample(prophet(x, y, N_frequencies), NUTS(), 1000, progress=PRINT_PROGRESS, init_params=map_estimate.values.array);

In [ ]:
map_vector = map_estimate.values.array
n_changepoints = length(y) ÷ 12

slope, intercept, error, tau,  adjustments = map_vector[1:4]..., map_vector[5:(5+n_changepoints-1)]
beta = map_vector[5+n_changepoints : end]

plot_prophet_model(air_passengers_2013_2023, slope, intercept, error, adjustments, N_frequencies, beta)

Thanks to our fully Bayesian approach, the uncertainty around our prediction is know very large,  
reflecting the fact that the model does not fit the data well anymore.

In [ ]:
n_year_forecast = 4
forecast = n_year_forecast * 12
x_forecast = collect(maximum(x) .+ (1:forecast)./12)

y_pred = sample_forecast(res_prophet_2023, changepoints, forecast, 10)
plot_forecast(x, y, x_forecast, y_pred)

We upgrade the model to specifically treat the years of the pandemic as one-off abnormalities.

In [ ]:
@model function prophet_covid(x::Vector{Float64}, y::Vector{Float64}, N_frequencies::Int)
    slope ~ Normal(0,3)
    intercept ~ Normal(6,3)
    error ~ InverseGamma(2,3)
    n_changepoints = Int(ceil(length(y) / 12)) # changepoint at each year
    tau ~ InverseGamma(1,1)
    adjustments ~ filldist(Laplace(0,tau+1e-5), n_changepoints)
    beta ~ filldist(Normal(0,1.), 2*N_frequencies)

    # UPGRADE: latent shock variables for the years 2021 to 2023
    shock_2020 ~ Normal(-8,1)
    shock_2021 ~ Normal(-4,1)
    shock_2022 ~ Normal(-2,1)

    k = slope
    m = intercept
    j = 1
    for i in eachindex(y)
        t = (x[i] - x[1])
        if i % 12 == 1
            k += adjustments[j]
            m -= t * adjustments[j]
            j += 1
        end
        s = seasonality_component(t, N_frequencies, beta)


        # UPGRADE: extra "intercept" variables for the years 2021 to 2023
        if 2020 ≤ x[i] && x[i] < 2021
            d = shock_2020
        elseif 2021 ≤ x[i] && x[i] < 2022
            d = shock_2021
        elseif 2022 ≤ x[i] && x[i] < 2023
            d = shock_2022
        else
            d = 0
        end

        y[i] ~ Normal(k * t + m + s + d, error+1e-5)
    end
end

Fit the model again. First MAP, then full posterior.

In [ ]:
N_frequencies = 3
x = air_passengers_2013_2023[!,"Date"]
y = air_passengers_2013_2023[!,"Total"]
changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]

Turing.Random.seed!(0)
map_estimate = optimize(prophet_covid(x, y, N_frequencies), MAP(), Optim.Options(iterations=10000))

Turing.Random.seed!(0)
res_prophet_covid = sample(prophet_covid(x, y, N_frequencies), NUTS(), 1000, progress=PRINT_PROGRESS, init_params=map_estimate.values.array);

In [ ]:
function shock_component(t::Float64, shocks::Vector{Float64})
    if 2020 ≤ t && t < 2021
        d = shocks[1]
    elseif 2021 ≤ t && t < 2022
        d = shocks[2]
    elseif 2022 ≤ t && t < 2023
        d = shocks[3]
    else
        d = 0
    end
end

function plot_prophet_covid_model(data, slope, intercept, error, adjustments, N_frequencies, beta, shocks)
    p = base_plot(data)
    x = data[!,"Date"]
    t_min = x[1]
    changepoints = x[(1:length(x)) .% 12 .== 1] .- x[1]
    plot!(p, t -> prophet_model_f(t - t_min, slope, intercept, changepoints, adjustments, N_frequencies, beta) + shock_component(t,shocks), color="red")

    vline!(changepoints[abs.(adjustments) .> 0.01] .+ t_min, linestyle=:dash, color="black")
    p
end

In [ ]:
map_vector = map_estimate.values.array
n_changepoints = length(y) ÷ 12

slope, intercept, error, tau, adjustments = map_vector[1:4]..., map_vector[5:(5+n_changepoints-1)]
beta = map_vector[5+n_changepoints : 5+n_changepoints + 2*N_frequencies-1]
shocks = map_vector[5+n_changepoints + 2*N_frequencies : end]

plot_prophet_covid_model(air_passengers_2013_2023, slope, intercept, error, adjustments, N_frequencies, beta, shocks)

Now, after explicetly modelling the pandemic, our uncertainty estimates get tighter again.

In [ ]:
n_year_forecast = 4
forecast = n_year_forecast * 12
x_forecast = collect(maximum(x) .+ (1:forecast)./12)

y_pred = sample_forecast(res_prophet_covid, changepoints, forecast, 10)
plot_forecast(x, y, x_forecast, y_pred)